# Performance analysis

## General

### Imports

In [ ]:
import copy
import matplotlib.pyplot as plt
import random
import sys
import time

In [ ]:
from blockchains.mark_1.blockchain import Blockchain as BlockchainNoPOW

## Insert

### General

#### Notes

I'll use `copy.deepcopy` for inserting a new elements (with a different id) in the data structure each time.  
Otherwise, Python by default will reuse and only reference the very same data structure - you can check this using `id()` on each element.  
`sys.getsizeof` is not deep: for complex data structures (such as the block, which is a `dict` containing a `list`) I need to add the weight of the single elements - taking the weight of the external structure will not include the internal elements.

### Common data structures: Dictionary, Set, List

#### Generating test data

In [ ]:
min_test_rounds = 1
max_test_rounds = 1002
step_test_round = 100
test_rounds = range(min_test_rounds, max_test_rounds, step_test_round)
test_data = 'Chancellor on brink of second bailout for banks'

##### Notes:
* I am using strings for the test because Python's set only accepts hashable elements

In [ ]:
def test_data_structures_without_POW_insert(test_rounds=test_rounds):
    # SUT_dictionary
    SUT_dictionary_insert_time_results = []
    SUT_dictionary_insert_weight_results = []

    # SUT_set
    SUT_set_insert_time_results = []
    SUT_set_insert_weight_results = []

    # SUT_list
    SUT_list_insert_time_results = []
    SUT_list_insert_weight_results = []


    for test_round in test_rounds:
        # SUT_dictionary
        # Reset the SUT
        SUT_dictionary = dict()
        
        start_time = time.time()
        for element in range(test_round):
            SUT_dictionary[element] = f'{test_data} {element}'
        end_time = time.time()
        SUT_dictionary_insert_time_results.append(end_time - start_time)
        SUT_dictionary_insert_weight_results.append(sys.getsizeof(SUT_dictionary))

        # SUT_set
        # Reset the SUT
        SUT_set = set()
        
        start_time = time.time()
        for element in range(test_round):
            SUT_set.add(f'{test_data} {element}')
        end_time = time.time()
        SUT_set_insert_time_results.append(end_time - start_time)
        SUT_set_insert_weight_results.append(sys.getsizeof(SUT_set))

        # SUT_list
        # Reset the SUT
        SUT_list = list()
        
        start_time = time.time()
        for element in range(test_round):
            SUT_list.append(f'{test_data} {element}')
        end_time = time.time()
        SUT_list_insert_time_results.append(end_time - start_time)
        SUT_list_insert_weight_results.append(sys.getsizeof(SUT_list))

    return (SUT_dictionary_insert_time_results,
            SUT_dictionary_insert_weight_results,
            SUT_set_insert_time_results,
            SUT_set_insert_weight_results,
            SUT_list_insert_time_results,
            SUT_list_insert_weight_results)

### Blockchain

#### Without POW

##### List implementation

In [ ]:
def test_blockchain_without_POW_insert(test_rounds=test_rounds):
    blockchain_list_sut = BlockchainNoPOW()
    blockchain_list_tests_insert_time_results = []
    blockchain_list_tests_insert_time_transaction_results = []
    blockchain_list_tests_insert_time_block_results = []
    blockchain_list_tests_insert_weight_results = []

    for test_round in test_rounds:
        # Reset the SUT
        blockchain_list_sut = BlockchainNoPOW()
        transaction_add_time = 0
        block_add_time = 0
        weight = 0
        
        for element in range(test_round):
            start_time_transaction = time.time()
            transaction = blockchain_list_sut.add_transaction(f'{test_data} {element}')
            start_time_block = time.time()
            block = blockchain_list_sut.mine()
            end_time_block = time.time()
            transaction_add_time += start_time_block - start_time_transaction
            block_add_time += end_time_block - start_time_block
            weight += sys.getsizeof(transaction)
            weight += sys.getsizeof(block)
        blockchain_list_tests_insert_time_results.append(transaction_add_time + block_add_time)
        blockchain_list_tests_insert_time_transaction_results.append(transaction_add_time)
        blockchain_list_tests_insert_time_block_results.append(block_add_time)
        blockchain_list_tests_insert_weight_results.append(weight)

    return (blockchain_list_tests_insert_time_results,
            blockchain_list_tests_insert_time_transaction_results,
            blockchain_list_tests_insert_time_block_results,
            blockchain_list_tests_insert_weight_results)

##### Set implementation

The blockchain should be implemented using the `set` Python data structure, but `set` only accepts hashable types. Using the list as data structure for the internal chain I could easily store even unhashable type in this simple implementation of the Blockhain.  
Also, `set` does not scale linearly in weight.

### Results

#### Data

In [ ]:
# Data Structure
(SUT_dictionary_insert_time_results,
 SUT_dictionary_insert_weight_results,
 SUT_set_insert_time_results,
 SUT_set_insert_weight_results,
 SUT_list_insert_time_results,
 SUT_list_insert_weight_results) = test_data_structures_without_POW_insert(test_rounds)
# Blockchain
(blockchain_list_tests_insert_time_results,
 blockchain_list_tests_insert_time_transaction_results,
 blockchain_list_tests_insert_time_block_results,
 blockchain_list_tests_insert_weight_results) = test_blockchain_without_POW_insert(test_rounds)

#### Time

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,10), sharex=True, sharey=True)
fig.suptitle('Data Insert', fontsize=20)

ylabel_text = 'Time'
xlabel_text = 'Number of blocks'

l1 = ax1.plot(test_rounds, SUT_dictionary_insert_time_results, label='Dictionary')[0]
l2 = ax1.plot(test_rounds, SUT_set_insert_time_results, label='Set')[0]
l3 = ax1.plot(test_rounds, SUT_list_insert_time_results, label='List')[0]
l4 = ax1.plot(test_rounds, blockchain_list_tests_insert_time_results, label='Blockchain')[0]
ax1.set_title('Global time for Data Structures and Blockchain')
ax1.set_ylabel(ylabel_text)
ax1.set_xlabel(xlabel_text)
ax1.legend()

l5 = ax2.plot(test_rounds, blockchain_list_tests_insert_time_transaction_results, label='Blockchain (Transaction)')[0]
l6 = ax2.plot(test_rounds, blockchain_list_tests_insert_time_block_results, label='Blockchain (Block)')[0]
l7 = ax2.plot(test_rounds, blockchain_list_tests_insert_time_results, label='Blockchain (Total)')[0]
ax2.set_title('Global time for Blockchain actions')
ax2.set_ylabel(ylabel_text)
ax2.set_xlabel(xlabel_text)
ax2.legend()

plt.show()

#### Weight

In [ ]:
nonlinear_test_rounds = range(1, 252, 50)

In [ ]:
def test_blockchain_without_POW_nonlinear_insert(test_rounds=test_rounds):
    blockchain_list_nonlinear_sut = BlockchainNoPOW()
    blockchain_list_nonlinear_tests_insert_time_results = []
    blockchain_list_nonlinear_tests_insert_time_transaction_results = []
    blockchain_list_nonlinear_tests_insert_time_block_results = []
    blockchain_list_nonlinear_tests_insert_weight_results = []

    for test_round in test_rounds:
        # Reset the SUT
        blockchain_list_nonlinear_sut = BlockchainNoPOW()
        transaction_add_time = 0
        block_add_time = 0
        weight = 0
        
        for element in range(test_round):
            # Upper limit between the (random int/coefficient) or random between 1 and limit
            coefficient = 10
            limit = random.randint(1, test_rounds[-1])
            less_than = bool(random.getrandbits(1))
            upper_limit = int(limit / coefficient) if less_than else random.randint(1,limit)
            
            start_time_transaction = time.time()
            for turn in range(upper_limit):
                transaction = blockchain_list_nonlinear_sut.add_transaction(f'{test_data} {turn}')
                weight += sys.getsizeof(transaction)
            start_time_block = time.time()
            block = blockchain_list_nonlinear_sut.mine()
            weight += sys.getsizeof(block)
            end_time_block = time.time()
            transaction_add_time += start_time_block - start_time_transaction
            block_add_time += end_time_block - start_time_block
        blockchain_list_nonlinear_tests_insert_time_results.append(transaction_add_time + block_add_time)
        blockchain_list_nonlinear_tests_insert_time_transaction_results.append(transaction_add_time)
        blockchain_list_nonlinear_tests_insert_time_block_results.append(block_add_time)
        blockchain_list_nonlinear_tests_insert_weight_results.append(weight)

    return (blockchain_list_nonlinear_tests_insert_time_results,
            blockchain_list_nonlinear_tests_insert_time_transaction_results,
            blockchain_list_nonlinear_tests_insert_time_block_results,
            blockchain_list_nonlinear_tests_insert_weight_results)

In [ ]:
# Blockchain linear insert
(blockchain_list_tests_insert_time_results,
 blockchain_list_tests_insert_time_transaction_results,
 blockchain_list_tests_insert_time_block_results,
 blockchain_list_tests_insert_weight_results) = test_blockchain_without_POW_insert(test_rounds=nonlinear_test_rounds)
# Blockchain nonlinear insert
(blockchain_list_nonlinear_tests_insert_time_results,
 blockchain_list_nonlinear_tests_insert_time_transaction_results,
 blockchain_list_nonlinear_tests_insert_time_block_results,
 blockchain_list_nonlinear_tests_insert_weight_results) = test_blockchain_without_POW_nonlinear_insert(test_rounds=nonlinear_test_rounds)

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(15,10), sharex=True, sharey=True)
fig.suptitle('Time', fontsize=20)

ylabel_text = 'Time'
xlabel_text = 'Number of blocks'

l1 = ax1.plot(nonlinear_test_rounds, blockchain_list_tests_insert_time_results, label='Blockchain (linear)')[0]
l2 = ax1.plot(nonlinear_test_rounds, blockchain_list_nonlinear_tests_insert_time_results, label='Blockchain (nonlinear)')[0]
ax1.set_title('Global time for Blockchain and Non Linear Blockchain')
ax1.set_ylabel(ylabel_text)
ax1.set_xlabel(xlabel_text)
ax1.legend()

plt.show()

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(15,10), sharex=True, sharey=True)
fig.suptitle('Weight', fontsize=20)

ylabel_text = 'Time'
xlabel_text = 'Number of blocks'

l1 = ax1.plot(nonlinear_test_rounds, blockchain_list_tests_insert_weight_results, label='Blockchain (linear)')[0]
l2 = ax1.plot(nonlinear_test_rounds, blockchain_list_nonlinear_tests_insert_weight_results, label='Blockchain (nonlinear)')[0]
ax1.set_title('Global weight for Blockchain and Non Linear Blockchain')
ax1.set_ylabel(ylabel_text)
ax1.set_xlabel(xlabel_text)
ax1.legend()

plt.show()

## Reading

Data are only accessible via the `get_chain()` method, no direct access.

## Updating

No updates allowed without corrupting the data structure or recreating all the blocks succeding the one you're changing.

# Result of the analysis

## Definition of Database

"A large amount of information stored in a computer system in such a way that it can be easily **looked at or changed**"  
 - [Cambridge Dictionary](https://dictionary.cambridge.org/dictionary/english/database)

## A Blockchain is not a Database

* Data can't be changed nor looked at (easily or by default)!

* Data are not stored efficiently (for insert, changes or lookup)

* A blockchain is not efficient and is not made for efficience!